experimental + custom components

In [ ]:
# 해당 셀을 실행한 후에 반드시 "런타임 다시시작"을 해주세요
!pip3 install kfp google-cloud-aiplatform --upgrade -q --user

In [ ]:
import kfp
from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output,
                        OutputPath, component, ClassificationMetrics, Metrics)
from kfp.v2.google import experimental

from google.cloud import aiplatform as vertex_ai

import tensorflow as tf

import time
from typing import Optional

## GCP 연결

In [ ]:
# gcp 연결
from google.colab import auth as google_auth

google_auth.authenticate_user()

# 환경변수 설정
아래 항목들을 본인 환경에 맞게 수정해주세요.

* PROJECT_ID = <프로젝트 ID>
* REGION = <리전>
* BUCKET_NAME = <bucket 이름>
* USER = <user 이름>

In [ ]:
# GCP 설정
PROJECT_ID = 'vertexai-pipeline'
REGION = "us-central1"
USER = "JeongMin-Do"
BUCKET = "vertexai-pipeline-bucket"
PIPELINE_NAME = "mnist"

# wandb 설정
WANDB_API_KEY = "135017ad091dd0ca4896f8f5c427e9f982c2e9a9"

In [ ]:
ts = int(time.time())
BUCKET_NAME = f"gs://{BUCKET}"
PIPELINE_ROOT = f"{BUCKET_NAME}/pipeline_root/{USER}"
WORKING_DIR = f"{PIPELINE_ROOT}/{ts}"
MODEL_DISPLAY_NAME = f"train_deploy{ts}"
MODEL_DIR = f"pipeline_root/{USER}/{ts}"

vertex_ai.init(project=PROJECT_ID)

# Pipeline 정의

#### 해당 부분을 요구에 맞게 수정해주세요

In [ ]:
@component
def train(training_state: str):
    print(f"training task: {training_state}")

@component(base_image="silverstar456/vertexai:aiplatform")
def model_upload(
    project: str,
    location: str,
    display_name: str,
    serving_container_image_uri: str,
    artifact_uri: Optional[str]=None,
):
    from google.cloud import aiplatform as vertex_ai

    vertex_ai.init(project=project, location=location)

    model = vertex_ai.Model.upload(
        display_name=display_name,
        artifact_uri=artifact_uri,
        serving_container_image_uri=serving_container_image_uri,
    )

@component(base_image="silverstar456/vertexai:aiplatform")
def model_deploy(
    project: str,
    location: str,
    display_name: str,
):
    from google.cloud import aiplatform as vertex_ai

    vertex_ai.init(project=project, location=location)
    model = vertex_ai.Model.list()[0]

    endpoint = vertex_ai.Endpoint.create(display_name=display_name)
    model.deploy(endpoint=endpoint, 
                 machine_type="n1-standard-2", 
                 min_replica_count=1, 
                 max_replica_count=1)

In [ ]:
@kfp.dsl.pipeline(name="mnist-pipeline-" + str(ts), pipeline_root=PIPELINE_ROOT)
def pipeline():
    # model train
    train_task = train("model training")
    experimental.run_as_aiplatform_custom_job(
        train_task,
        worker_pool_specs=[
            {
                "containerSpec": {
                    "env": [{"name": "WORKING_DIR", "value": MODEL_DIR},
                            {"name": "BUCKET", "value": BUCKET}],
                    "imageUri": "silverstar456/vertexai:mnist-pipeline", # 사전에 빌드한 도커 이미지 URI
                },
                "replicaCount": "1",
                "machineSpec": {
                    "machineType": "n1-standard-8", # 실행시 사용할 머신 타입 GCP 문서 참조
                    "accelerator_type": vertex_ai.gapic.AcceleratorType.NVIDIA_TESLA_V100, # GPU 타입
                    "accelerator_count": 1, # GPU 개수
                },
            }
        ],
    )

    # model upload
    uploader = model_upload(
        project=PROJECT_ID,
        location=REGION,
        display_name=MODEL_DISPLAY_NAME,
        artifact_uri=WORKING_DIR,
        serving_container_image_uri="gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest"
        )
    uploader.after(train_task)
    
    # model deploy
    deployer = model_deploy(
        project=PROJECT_ID,
        location=REGION,
        display_name=MODEL_DISPLAY_NAME,
    )
    deployer.after(uploader)

In [ ]:
compiler.Compiler().compile(
    pipeline_func = pipeline, package_path = "mnist-pipeline.json"
)

vertex_ai.init(project=PROJECT_ID)
pipeline = vertex_ai.PipelineJob(
    display_name="mnist",
    template_path="mnist-pipeline.json",
    parameter_values={},
)

pipeline.run(sync=False)

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
